### 内容
1. 嵌入模型私有化
2. 向量数据库私有化
3. 大语言模型私有化

### 私有化，肯定要涉及到基础设施的部署
通常是五部曲
1. 购买硬件/云服务器（Dell、IBM、Cloud..）
3. 安装操作系统（Ubuntu、CentOS..）
4. 打上显卡驱动（NVIDIA Drive）
5. 选择算法框架及计算架构 （PyTorch-Meta, TensorFlow-Google, MindSpore-Huawei...）（CUDA）
6. 运行对应的模型（Embedding、Language...）

In [ ]:
# 问题：无法下载到模型权重

# 方式一：https://huggingface.co/ HuggingFace Mirror
# https://aliendao.cn/

# 方式二：国内平替，例如阿里云摩搭
# https://www.modelscope.cn/

In [ ]:
# 问题：没有本地算力资源
# 方式一：用第三方租用，例如 AWS、腾讯云的 GPU 实例
# 方式二：第三方托管（开源模型），例如阿里云灵积直接调用 API（某种意义上也可以做到可控）
# https://dashscope.console.aliyun.com/

### 第一架马车：Embedding Model

In [ ]:
# 开源模型有很多，也可以自己来根据业务训练（例如专注于零售、医疗、供应链领域..）
# 选择一个性能不错的，可以开箱即用的 (Moka Massive Mixed Embedding)
# https://huggingface.co/moka-ai/m3e-base

In [23]:
# 很简单的工作过程，输入n个句子，给回n个d维度的向量

from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(model_name_or_path="./weights/embedding/m3e-base/")
sentence = ['How are you', "你好", "Bonjour", "こんにちは"]
vectors = embed_model.encode(sentence)

print(f'提供有 {len(vectors)} 个向量，每个向量有 {len(vectors[0])} 维度')

提供有 4 个向量，每个向量有 768 维度


In [4]:
vectors.shape

(4, 768)

In [5]:
vectors

array([[ 1.1611574 ,  0.37050796,  0.7901164 , ..., -0.6740078 ,
        -1.0759622 , -0.6462538 ],
       [ 0.5670725 ,  0.44698715,  1.1844376 , ..., -1.0427248 ,
        -0.8064798 , -0.24840572],
       [ 0.43304953,  0.79238814,  0.9573602 , ..., -1.1035961 ,
        -0.23207028, -0.58914644],
       [ 0.4566425 ,  0.63839805,  1.0179789 , ..., -1.3621017 ,
        -1.0953146 , -0.644257  ]], dtype=float32)

In [9]:
# 不同的模型，有不同的维度，维度越多，能表达的信息量越多
# 同一个模型，也有不同的尺寸，例如 m3e 中的 small, base, large 

from sentence_transformers import SentenceTransformer
m3e_small = SentenceTransformer(model_name_or_path="./weights/embedding/m3e-small/")
vectors = m3e_small.encode("Hello")
vectors.shape

(512,)

### 第二架马车：Vector Database

In [ ]:
# 用传统的数据库来做存储功能也可以，但是基于向量查询就需要在程序内存中处理
# 支持私有化部署，对于扩展性、可用性、性能都有高要求 （Milvus）
# https://milvus.io/docs

In [11]:
# 增删查改来一套
from pymilvus import connections, utility, Collection, db, CollectionSchema, FieldSchema, DataType

connections = connections.connect(host="192.168.0.221", port=19530, user="root", password="")

In [12]:
# 默认系统会自动创建一个名为 default 的数据库
database_name = "motang"
# 创建一个数据库
db.create_database(database_name)
# 查看实例下有多少个数据库
db.list_database()
# 删库跑路
db.drop_database(database_name)

In [13]:
# Collection 类似传统数据库中的 Table 概念
database_name = "default"
collection_name = "mobot"

In [42]:
# 创建集合

# 1. 声明 Field 类型及属性
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="answer", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768)
]

# 2. 根据字段声明 Schema
schema = CollectionSchema(
    fields=fields,
    description="",
    enable_dynamic_field=True
)

# 3. 根据 Schema 声明 Collection
collection = Collection(
    name=collection_name,
    schema=schema,
    using=database_name,
)

# 4. 创建 Field 的索引
index_params = {
    'index_type': "IVF_FLAT",
    'metric_type': "L2",
    'params': {'nlist': 1}
}
collection.create_index(field_name="embedding", index_params=index_params)

# 5. 挂载 Collection，变成可用状态
collection.load()

In [21]:
# 查看 Collection 详情
Collection(collection_name)

<Collection>:
-------------
<name>: mobot
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'question', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'answer', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}], 'enable_dynamic_field': True}

In [41]:
# 删库大法
utility.drop_collection(collection_name)

In [18]:
# 判断 Collection 是否存在
utility.has_collection(collection_name)

False

#### Document

In [43]:
# 新增数据
knowledge_base = [
    {"question": "模利是谁", "answer": "模利是丰速科技的架构师，全名为唐模利。"},
    {"question": "老牛是谁", "answer": "老牛是丰速科技的人财中心负责人，全名是牛总。"},
    {"question": "易打单是什么", "answer": "是一款电商打运单软件，各大电商平台商家订单、运单协同处理SaaS平台，集成了订单管理、运单打印、一键发货、单号分享、物流匹配、查询路由、运费对账、商品管理、库存管理、开放平台等功能，对接主流电商平台超过40家，可打印70+主流快递公司的电子面单，为商家提供方便快捷的服务。"}

]

ids, questions, answers, vectors = [], [], [], []

for i in range(len(knowledge_base)):
    ids.append(i+1)
    questions.append(knowledge_base[i]['question'])
    answers.append(knowledge_base[i]['answer'])
    vectors.append(embed_model.encode(f'{knowledge_base[i]["question"]}\n\n{knowledge_base[i]["answer"]}'))


entities = [ids, questions, answers, vectors]

collection.insert(entities)
collection.flush()

In [47]:
# 基于向量的相似度搜索

data = embed_model.encode(["老牛是谁"])
top_k = 3

documents = collection.search(
    data=data,
    anns_field="embedding",
    param={"metric_type": "L2", "params": {"nprobe": 1}, "offset": 0},
    limit=top_k,
    output_fields=["*"]
)

In [48]:
for i in range(len(documents[0])):
    print(documents[0][i].id)
    print(documents[0][i].distance)
    print(documents[0][i].entity.get('question'))
    print(documents[0][i].entity.get('answer'))
    print('----------------')

2
94.0130615234375
老牛是谁
老牛是丰速科技的人财中心负责人，全名是牛总。
----------------
1
231.83514404296875
模利是谁
模利是丰速科技的架构师，全名为唐模利。
----------------
3
303.7200927734375
易打单是什么
是一款电商打运单软件，各大电商平台商家订单、运单协同处理SaaS平台，集成了订单管理、运单打印、一键发货、单号分享、物流匹配、查询路由、运费对账、商品管理、库存管理、开放平台等功能，对接主流电商平台超过40家，可打印70+主流快递公司的电子面单，为商家提供方便快捷的服务。
----------------


In [49]:
# 基于标量的查询
expr = 'id != 3'

documents = collection.query(
    expr=expr,
    offset=0,
    limit=top_k,
    output_fields=["*"]
)

In [50]:
for i in range(len(documents)):
    print(documents[i]['id'])
    print(documents[i]['question'])
    print(documents[i]['answer'])
    print('----------------')

1
模利是谁
模利是丰速科技的架构师，全名为唐模利。
----------------
2
老牛是谁
老牛是丰速科技的人财中心负责人，全名是牛总。
----------------


In [52]:
# 混合检索
data = embed_model.encode(["易打单"])
expr = 'id != 3'

documents = collection.search(
    data=data,
    anns_field="embedding",
    param={"metric_type": "L2", "params": {"nprobe": 1}, "offset": 0},
    limit=top_k,
    output_fields=["*"],
    expr=expr
)

In [53]:
for i in range(len(documents[0])):
    print(documents[0][i].id)
    print(documents[0][i].distance)
    print(documents[0][i].entity.get('question'))
    print(documents[0][i].entity.get('answer'))
    print('----------------')

2
263.9797668457031
老牛是谁
老牛是丰速科技的人财中心负责人，全名是牛总。
----------------
1
280.9371337890625
模利是谁
模利是丰速科技的架构师，全名为唐模利。
----------------


In [54]:
# 删除前先查下是否存在
expr = 'id in [3]'

documents = collection.query(
    expr=expr,
    offset=0,
    limit=top_k,
    output_fields=["*"]
)

print(len(documents))

1


In [35]:
# 删除数据
collection.delete(expr)

(insert count: 0, delete count: 1, upsert count: 0, timestamp: 445019755041259522, success count: 0, err count: 0)

In [55]:
# 再查一下，看看是否还存在
documents = collection.query(
    expr=expr,
    offset=0,
    limit=top_k,
    output_fields=["*"]
)

print(len(documents))

1
